For this homework we will be using the Yellow 2024-10 data from the official website: 

```bash
wget https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-10.parquet
```

## Question 1: Install Spark and PySpark

- Install Spark
- Run PySpark
- Create a local spark session
- Execute spark.version.

What's the output?

### Answer 

- See two cells below

In [1]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import types
from pyspark.sql import functions as F

spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


25/06/03 17:00:09 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
pyspark.__version__




'3.3.2'

## Question 2: Yellow October 2024

Read the October 2024 Yellow into a Spark Dataframe.

Repartition the Dataframe to 4 partitions and save it to parquet.

What is the average size of the Parquet (ending with .parquet extension) Files that were created (in MB)? Select the answer which most closely matches.

### Answer

- 25MB

In [3]:
!wget https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-10.parquet

--2025-06-03 17:00:17--  https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-10.parquet
Resolving d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)... 18.239.238.152, 18.239.238.212, 18.239.238.133, ...
Connecting to d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)|18.239.238.152|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 64346071 (61M) [binary/octet-stream]
Saving to: ‘yellow_tripdata_2024-10.parquet.1’

yellow_tripdata_202 100%[===================>]  61.36M   225MB/s    in 0.3s    

2025-06-03 17:00:17 (225 MB/s) - ‘yellow_tripdata_2024-10.parquet.1’ saved [64346071/64346071]



In [4]:
df_yellow = spark.read.parquet('yellow_tripdata_2024-10.parquet')

In [5]:
df_yellow.printSchema()


root
 |-- VendorID: integer (nullable = true)
 |-- tpep_pickup_datetime: timestamp (nullable = true)
 |-- tpep_dropoff_datetime: timestamp (nullable = true)
 |-- passenger_count: long (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RatecodeID: long (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- payment_type: long (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)
 |-- Airport_fee: double (nullable = true)



In [6]:
df_yellow = df_yellow.repartition(4)
df_yellow.write.parquet('data-out')

In [7]:
ls -lh data-out

total 100M
-rw-r--r-- 1 willwaight394 willwaight394   0 Jun  3 17:03 _SUCCESS
-rw-r--r-- 1 willwaight394 willwaight394 25M Jun  3 17:03 part-00000-245e3b93-8167-40e8-a04a-48ce46c0cbc0-c000.snappy.parquet
-rw-r--r-- 1 willwaight394 willwaight394 25M Jun  3 17:03 part-00001-245e3b93-8167-40e8-a04a-48ce46c0cbc0-c000.snappy.parquet
-rw-r--r-- 1 willwaight394 willwaight394 25M Jun  3 17:03 part-00002-245e3b93-8167-40e8-a04a-48ce46c0cbc0-c000.snappy.parquet
-rw-r--r-- 1 willwaight394 willwaight394 25M Jun  3 17:03 part-00003-245e3b93-8167-40e8-a04a-48ce46c0cbc0-c000.snappy.parquet


## Question 3: Count records 

How many taxi trips were there on the 15th of October?

Consider only trips that started on the 15th of October.

### Answer

- 128,893

In [9]:
df_yellow.take(5)

[Row(VendorID=2, tpep_pickup_datetime=datetime.datetime(2024, 10, 4, 23, 19, 27), tpep_dropoff_datetime=datetime.datetime(2024, 10, 4, 23, 43, 28), passenger_count=1, trip_distance=6.07, RatecodeID=1, store_and_fwd_flag='N', PULocationID=229, DOLocationID=231, payment_type=1, fare_amount=31.0, extra=1.0, mta_tax=0.5, tip_amount=4.0, tolls_amount=0.0, improvement_surcharge=1.0, total_amount=40.0, congestion_surcharge=2.5, Airport_fee=0.0),
 Row(VendorID=2, tpep_pickup_datetime=datetime.datetime(2024, 10, 12, 22, 54, 1), tpep_dropoff_datetime=datetime.datetime(2024, 10, 12, 23, 18, 59), passenger_count=1, trip_distance=10.15, RatecodeID=1, store_and_fwd_flag='N', PULocationID=138, DOLocationID=25, payment_type=1, fare_amount=41.5, extra=6.0, mta_tax=0.5, tip_amount=9.8, tolls_amount=0.0, improvement_surcharge=1.0, total_amount=60.55, congestion_surcharge=0.0, Airport_fee=1.75),
 Row(VendorID=2, tpep_pickup_datetime=datetime.datetime(2024, 10, 12, 17, 7, 34), tpep_dropoff_datetime=datetim

In [ ]:
df_yellow.createOrReplaceTempView('october_yellow_taxi_trips')

> *Note: Must create a temporary view in order to reference schema for sql query*

In [ ]:
spark.sql("""
SELECT
    count(1) as num_trips
FROM
    october_yellow_taxi_trips
WHERE
    CAST(tpep_pickup_datetime as DATE) = '2024-10-15'
AND 
    tpep_dropoff_datetime IS NOT NULL
""").show()

+---------+
|num_trips|
+---------+
|   128893|
+---------+



## Question 4: Longest trip

What is the length of the longest trip in the dataset in hours?

### Answer 

- 162 hours

In [18]:
df_yellow_longest = df_yellow.withColumn('duration', (df_yellow.tpep_dropoff_datetime.cast('long') - df_yellow.tpep_pickup_datetime.cast('long'))/60/60) \
.select('duration') \
.orderBy('duration', ascending=False) \
.limit(5) \
.show()


+------------------+
|          duration|
+------------------+
| 162.6177777777778|
|           143.325|
|137.76055555555556|
|114.83472222222221|
| 89.89833333333333|
+------------------+



## Question 5: User Interface

Spark’s User Interface which shows the application's dashboard runs on which local port?

### Answer 

- 4040


## Question 6: Least frequent pickup location zone

Load the zone lookup data into a temp view in Spark:

```bash
wget https://d37ci6vzurychx.cloudfront.net/misc/taxi_zone_lookup.csv
```

Using the zone lookup data and the Yellow October 2024 data, what is the name of the LEAST frequent pickup location Zone?

### Answer 

- Both 'Freshkills Park' and 'Great Kills Park' have only one trip each.

In [20]:
!wget https://d37ci6vzurychx.cloudfront.net/misc/taxi_zone_lookup.csv

--2025-06-03 17:21:45--  https://d37ci6vzurychx.cloudfront.net/misc/taxi_zone_lookup.csv
Resolving d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)... 18.239.238.119, 18.239.238.152, 18.239.238.133, ...
Connecting to d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)|18.239.238.119|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 12331 (12K) [text/csv]
Saving to: ‘taxi_zone_lookup.csv.1’

taxi_zone_lookup.cs 100%[===================>]  12.04K  --.-KB/s    in 0s      

2025-06-03 17:21:45 (116 MB/s) - ‘taxi_zone_lookup.csv.1’ saved [12331/12331]



In [27]:
df_zones_least = spark.read.option("header", "true").csv('taxi_zone_lookup.csv.1')

> *Note: Must specify option to include headers in order to clearly in reference in sql query*

In [28]:
df_zones_least.show()

+----------+-------------+--------------------+------------+
|LocationID|      Borough|                Zone|service_zone|
+----------+-------------+--------------------+------------+
|         1|          EWR|      Newark Airport|         EWR|
|         2|       Queens|         Jamaica Bay|   Boro Zone|
|         3|        Bronx|Allerton/Pelham G...|   Boro Zone|
|         4|    Manhattan|       Alphabet City| Yellow Zone|
|         5|Staten Island|       Arden Heights|   Boro Zone|
|         6|Staten Island|Arrochar/Fort Wad...|   Boro Zone|
|         7|       Queens|             Astoria|   Boro Zone|
|         8|       Queens|        Astoria Park|   Boro Zone|
|         9|       Queens|          Auburndale|   Boro Zone|
|        10|       Queens|        Baisley Park|   Boro Zone|
|        11|     Brooklyn|          Bath Beach|   Boro Zone|
|        12|    Manhattan|        Battery Park| Yellow Zone|
|        13|    Manhattan|   Battery Park City| Yellow Zone|
|        14|     Brookly

In [29]:
df_zones_least.createOrReplaceTempView("least_frequent_zones")

In [ ]:
spark.sql("""
SELECT
    lfz.Zone, 
    count(*) as num_trips
FROM
    least_frequent_zones as lfz
LEFT JOIN
    october_yellow_taxi_trips as oct
ON 
    lfz.LocationID = oct.PULocationID
GROUP BY 
    1
ORDER BY 
    2 ASC
""").take(5)

[Row(Zone='Freshkills Park', num_trips=1),
 Row(Zone='Great Kills Park', num_trips=1),
 Row(Zone='Rikers Island', num_trips=2),
 Row(Zone='Arden Heights', num_trips=2),
 Row(Zone="Governor's Island/Ellis Island/Liberty Island", num_trips=3)]